In [97]:
import numpy as np
import pandas as pd

from itertools import chain
from collections import defaultdict

In [9]:
df_title = pd.read_csv('../data/cb12/processed/df_title_MinorGroup200_tokenized.csv', sep=';')
print(df_title.shape)
print("Number of unique JobTitles (raw): ", len(df_title.JobTitle.unique()))
print("Number of unique tokenized JobTitles: ", len(df_title.JobTitle_token.unique()))
print("Number of unique Users: ", len(df_title.UserId.unique()))

(18088, 8)
Number of unique JobTitles (raw):  9219
Number of unique tokenized JobTitles:  9216
Number of unique Users:  4907


In [45]:
vocab_lemma_filtered = set(chain(chain.from_iterable([eval(x) for x in df_title['JobTitle_token_Id']])))
print('Vocab size: {}'.format(len(vocab_lemma_filtered)))

Vocab size: 1682


# Step1: Create file

### titles.csv

In [98]:
Ids = []
JobTitle_tokens = []
JobTitle_tokens_idx = []
MajorGroups = []
MinorGroups = []
DetailedOccupations = []

Id = 0
for idx, row in df_title.iterrows():
    if row["JobTitle_token"] not in JobTitle_tokens:
        Ids.append(Id)
        JobTitle_tokens.append(row["JobTitle_token"])
        JobTitle_tokens_idx.append(row["JobTitle_token_Id"])
        MajorGroups.append(row["MajorGroup"])
        MinorGroups.append(row["MinorGroup"])
        DetailedOccupations.append(row["DetailedOccupation"])
        Id+=1

In [102]:
titles = pd.DataFrame({"Id": Ids, 
                       "MajorGroup": MajorGroups, 
                       "MinorGroup": MinorGroups, 
                       "DetailedOccupation": DetailedOccupations, 
                       "JobTitle_tokens": JobTitle_tokens,
                       "JobTitle_tokens_idx": JobTitle_tokens_idx})
print(titles.shape)
titles.to_csv('../data/cb12/graph/titles.csv', sep='\t', index=False)

(9216, 6)


In [48]:
dict_JobTitle_tokens2Id = dict(zip(JobTitle_tokens, Ids))

In [49]:
print(len(titles.DetailedOccupation.unique()))
print(len(titles.MinorGroup.unique()))
print(len(titles.MajorGroup.unique()))

300
32
16


### title_title_transition.csv
* transition
* add self-loop

In [100]:
Weight = defaultdict(int)

for filename, rows in df_title.groupby('UserId'):
    titles_thisresume = rows['JobTitle_token'].tolist()
    for i in range(len(titles_thisresume)):
        if i < len(titles_thisresume)-1:
            src = dict_JobTitle_tokens2Id[titles_thisresume[i]]
            dst = dict_JobTitle_tokens2Id[titles_thisresume[i+1]]
            Weight[(src,dst)] += 1


print(len(Weight))
print(np.sum(list(Weight.values())))
print(len([v for k, v in Weight.items() if k[0]==k[1]])/ len(Weight))


for idx, row in titles.iterrows():
    job = row["Id"]
    if (job,job) not in Weight:
        Weight[(job,job)] += 1
                                  
    
print(len(Weight))
print(np.sum(list(Weight.values())))

Src = [edge[0] for edge in Weight.keys()]
Dst = [edge[1] for edge in Weight.keys()]

11996
13181
0.04768256085361787
20640
21825


In [103]:
title_title_transition = pd.DataFrame({"Src": Src, "Dst": Dst, "Weight": list(Weight.values())})
title_title_transition.to_csv('../data/cb12/graph/title_title_transition_MinorGroup200.csv', sep='\t', index=False)

### title_title_transition_enhanced.csv
* if two titles have the same tag, there is a bi-directional edge between them
* add self-loop

In [90]:
f_in = open('../data/cb12/processed/Tags_MinorGroup200_200.txt', 'r')
tags = f_in.readlines()
tags = [t.strip('\n') for t in tags]
f_in.close()

In [93]:
Weight = defaultdict(int)
for filename, rows in df_title.groupby('UserId'):
    titles_thisresume = rows['JobTitle_token'].tolist()
    for i in range(len(titles_thisresume)):
        if i < len(titles_thisresume)-1:
            src = dict_JobTitle_tokens2Id[titles_thisresume[i]]
            dst = dict_JobTitle_tokens2Id[titles_thisresume[i+1]]
            Weight[(src,dst)] += 1


print(len(Weight))
print(np.sum(list(Weight.values())))
print(len([v for k, v in Weight.items() if k[0]==k[1]])/ len(Weight))

list_JobTitle_tokens = dict_JobTitle_tokens2Id.keys()
for tag in tags:
    list_JobTitle_hastag = [title for title in list_JobTitle_tokens if tag in title.split()]
    for i in range(len(list_JobTitle_hastag)):
        for j in range(len(list_JobTitle_hastag)):
            src = dict_JobTitle_tokens2Id[list_JobTitle_hastag[i]]
            dst = dict_JobTitle_tokens2Id[list_JobTitle_hastag[j]]
            Weight[(src,dst)] += 1

    
print(len(Weight))
print(np.sum(list(Weight.values())))


for idx, row in titles.iterrows():
    job = row["Id"]
    if (job,job) not in Weight:
        Weight[(job,job)] += 1
                                  
    
print(len(Weight))
print(np.sum(list(Weight.values())))


Src = [edge[0] for edge in Weight.keys()]
Dst = [edge[1] for edge in Weight.keys()]

11996
13181
0.04768256085361787
6477442
7133880
6477819
7134257


In [104]:
title_title_transition = pd.DataFrame({"Src": Src, "Dst": Dst, "Weight": list(Weight.values())})
title_title_transition.to_csv('../data/cb12/graph/title_title_transition_MinorGroup200_enhanced.csv', sep='\t', index=False)